In [98]:
import requests
from bs4 import BeautifulSoup
# Make a request to the website
r = requests.get('https://www.bbc.com')
# Create an object to parse the HTML format
soup = BeautifulSoup(r.content, 'html.parser')
# Retrieve all popular news links (Fig. 1)
link = []
for i in soup.find('div', {'class':"module__content"}).find_all('a'):
    i['href'] = "https://www.bbc.com"+i['href'] + '?page=all'
    link.append(i['href'])
new_link = []
for i in range(0, len(link), 3):
    new_link.append(link[i])
print(new_link)
del new_link[3:4]
print(new_link)
# For each link, we retrieve paragraphs from it, combine each paragraph as one string, and save it to documents (Fig. 2)
documents = []
for i in new_link:
    # Make a request to the link
    r = requests.get(i)
  
    # Initialize BeautifulSoup object to parse the content 
    soup = BeautifulSoup(r.content, 'html.parser')
  
    # Retrieve all paragraphs and combine it as one
    sen = []
    for i in soup.find('div', {'class':'ssrcss-7uxr49-RichTextContainer e5tfeyi1'}).find_all('p'):
        sen.append(i.text)

    # Add the combined paragraphs to documents
    documents.append(' '.join(sen))
    print(documents)

['https://www.bbc.com/news/world-europe-64102826?page=all', 'https://www.bbc.com/news/world-us-canada-64103825?page=all', 'https://www.bbc.com/news/world-asia-64106226?page=all', 'https://www.bbc.com/sport/football/64106065?page=all', 'https://www.bbc.comhttps://www.bbc.com/culture/article/20221221-avatar-2-what-is-the-future-for-special-effects?page=all']
['https://www.bbc.com/news/world-europe-64102826?page=all', 'https://www.bbc.com/news/world-us-canada-64103825?page=all', 'https://www.bbc.com/news/world-asia-64106226?page=all', 'https://www.bbc.comhttps://www.bbc.com/culture/article/20221221-avatar-2-what-is-the-future-for-special-effects?page=all']
["Thirteen-year-old Nika Selivanova made a heart shape with both her hands, waving goodbye to her best friend Inna who was pressed up against the glass partition that divided the entrance hall of Kherson's train station from the waiting area."]
["Thirteen-year-old Nika Selivanova made a heart shape with both her hands, waving goodbye to

ConnectionError: HTTPSConnectionPool(host='www.bbc.comhttps', port=443): Max retries exceeded with url: //www.bbc.com/culture/article/20221221-avatar-2-what-is-the-future-for-special-effects?page=all (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f9f21ca86d0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [99]:
import re
import string
documents_clean = []
for d in documents:
    # Remove Unicode
    document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
    # Remove Mentions
    document_test = re.sub(r'@\w+', '', document_test)
    # Lowercase the document
    document_test = document_test.lower()
    # Remove punctuations
    document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
    # Lowercase the numbers
    document_test = re.sub(r'[0-9]', '', document_test)
    # Remove the doubled space
    document_test = re.sub(r'\s{2,}', ' ', document_test)
    documents_clean.append(document_test)

In [100]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
# Instantiate a TfidfVectorizer object
vectorizer = TfidfVectorizer()
# It fits the data and transform it as a vector
X = vectorizer.fit_transform(documents_clean)
# Convert the X as transposed matrix
X = X.T.toarray()
# Create a DataFrame and set the vocabulary as the index
df = pd.DataFrame(X, index=vectorizer.get_feature_names())
df

,0,1,2
afghanistan,0.000000,0.0,0.272109
against,0.157226,0.0,0.000000
and,0.000000,0.0,0.272109
area,0.157226,0.0,0.000000
best,0.157226,0.0,0.000000
...,...,...,...
waving,0.157226,0.0,0.000000
who,0.157226,0.0,0.000000
with,0.157226,0.0,0.000000
women,0.000000,0.0,0.272109


In [102]:
def get_similar_articles(q, df):
  import numpy as np
  print("query:", q)
  print("The following are articles with the highest cosine similarity values: ")
    # Convert the query become a vector
  q = [q]
  q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
  sim = {}
    # Calculate the similarity
  for i in range(3):
     sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
    
    # Sort the values 
  sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
    # Print the articles and their similarity values
  for k, v in sim_sorted:
    if v != 0.0:
      print("Similarity Value:", v)
      print(documents_clean[k])        
      print()
  # Add The Query


q1 = input(str)
  # Call the function
get_similar_articles(q1, df)

query: and
The following are articles with the highest cosine similarity values: 
Similarity Value: 0.27210883045499096
the un security council has denounced taliban government policies targeting women and girls in afghanistan 

